## Treating a Function like an Object


In [1]:
def factorial(n):
    '''returns n!'''# __doc__
    return 1 if n < 2 else n * factorial(n-1)

factorial.__doc__, type(factorial)

('returns n!', function)

In [2]:
fact = factorial # Use function through a different name
list(map(fact, range(11))) # pass function as argument

[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]

## Higher-Order Functions
A function that takes a function as argument or returns a function as the result is a higher-order function. e.g. `map` and `sorted`

In [3]:
def reverse(word):
    return word[::-1]

fruits = ['strawberry', 'fig', 'apple', 'cherry', 'raspberry', 'banana']

sorted(fruits, key=reverse)

['banana', 'apple', 'fig', 'raspberry', 'strawberry', 'cherry']

### Modern replacements for old high-order functions
A listcomp or a genexp does the job of `map` and `filter` combined, but is more readable. 

`map` and `filter` return generators like [genexps](./02_An_Array_of_Sequences.ipynb)

In [4]:
print(list(map(fact, range(6))))
print([fact(n) for n in range(6)])

[1, 1, 2, 6, 24, 120]
[1, 1, 2, 6, 24, 120]


In [5]:
print(list(map(factorial, filter(lambda n: n % 2, range(6)))))
print([factorial(n) for n in range(6) if n % 2])

[1, 6, 120]
[1, 6, 120]


## Anonymous Functions
The `lambda` keyword creates an anonymous function within a Python expression. The best use of anonymous functions is in the context of an argument list for a higher-order function.

In [6]:
fruits = ['strawberry', 'fig', 'apple', 'cherry', 'raspberry', 'banana']

sorted(fruits, key=lambda word: word[::-1])

['banana', 'apple', 'fig', 'raspberry', 'strawberry', 'cherry']

## The Nine Flavors of Callable Objects

To determine whether an object is callable, use the `callable()` built-in function. 

Nine callable types:

1. **User-defined functions** created with `def` statements or `lambda` expressions.
2. **Built-in functions**, like `len` or `time.strftime`
3. **Built-in methods**, like `dict.get`
4. **Methods**. Functions defined in the body of a class.
5. **Classes**. When invoked, a class runs its `__new__` method to create an instance, then `__init__` to initialize it, and finally the instance is returned to the caller
6. **Class instances**. If a class defines a `__call__` method, then its instances may be invoked as functions
7. **Generator functions**. Functions or methods that use the `yield` keyword in their body. When called, they return a generator object.
8. **Native coroutine functions**. Functions or methods defined with `async def`
9. **Asynchronous generator functions**. Functions or methods defined with async def that have yield in their body.

In [7]:
[callable(obj) for obj in (abs, str, 13)]

[True, True, False]

### User-Defined Callable Types


In [8]:
import random

class BingoCage:

    def __init__(self, items):
        self._items = list(items)  
        random.shuffle(self._items)

    def pick(self): 
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')  

    def __call__(self): 
        return self.pick()

bingo = BingoCage(range(30))
bingo(), bingo() # implictly call `bingo.pick()`

(25, 7)

### Function Introspection
See what the `dir` function reveals about our `factorial`

In [9]:
dir(factorial)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

## From Positional to Keyword-Only Parameters
Keyword-Only Parameters can only be given as a keyword argument—it will **never** capture unnamed positional arguments. To specify keyword-only arguments when defining a function, name them **after** the argument prefixed with `*`

In [10]:
def f(a, *args, b=4):
    print(a, b)

f(1, 2, 3)
f(1, b=2)

1 4
1 2


Positional-only parameters can **only** be called with positional parameters, 

New in Python 3.8: To define a function requiring positional-only parameters, use `/` in the parameter list.

All arguments to the left of the `/` are positional-only. After the `/`, you may specify other arguments, which work as usual.

In [11]:
def divmod(a, b, /):
    print(a, b)

x, y = 1, 2
try:
    divmod(a=x, b=y)
except TypeError as e:
    print(e)
    divmod(x, y)

divmod() got some positional-only arguments passed as keyword arguments: 'a, b'
1 2


Within a function object, the `__defaults__` attribute holds a tuple with the default values of positional and keyword arguments. The defaults for keyword-only arguments appear in `__kwdefaults__`. The names of the arguments, however, are found within the `__code__` attribute

In [12]:
def clip(text, max_len=80):
    """Return text clipped at the last space before or after max_len
    """
    end = None
    if len(text) > max_len:
        space_before = text.rfind(' ', 0, max_len)
        if space_before >= 0:
            end = space_before
        else:
            space_after = text.rfind(' ', max_len)
            if space_after >= 0:
                end = space_after
    if end is None:  # no spaces were found
        return text.rstrip()
    return text[:end].rstrip()

In [13]:
print(clip.__defaults__) # need to scan fron last to first to tell which attribute this default value belongs to
print(clip.__code__.co_varnames) # also includes the names of the local variables created in the body of the function, arguments always at first
print(clip.__code__.co_argcount) # number of arguments, does not include any variable arguments prefixed with * or **

(80,)
('text', 'max_len', 'end', 'space_before', 'space_after')
2


In the example, we have two (according to `.__code__.co_argcount`) arguments, `text` and `max_len` (according to `.__code__.co_varnames`), and one default, 80 (according to `.__defaults__`), so it must belong to the last argument, `max_len`. This is awkward.

Fortunately, there is a better way: the `inspect` module.

In [14]:
from inspect import signature

sig = signature(clip)
for name, param in sig.parameters.items():
    print(param.kind, ':', name, '=', param.default)

POSITIONAL_OR_KEYWORD : text = <class 'inspect._empty'>
POSITIONAL_OR_KEYWORD : max_len = 80
